In [12]:
import pandas as pd
import csv
from sklearn import preprocessing
import numpy as np

In [36]:
def distance(x, y, cols=['BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES', 'ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE', 'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY', 'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY', 'CASH_ADVANCE_TRX', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS', 'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE']):
    
    return np.linalg.norm(x[cols].values - y[cols].values, axis=1)

def find_centers(df, k):
    centers = []
    first = df.sample()
    centers.append(first["CUST_ID"].values[0])
    print(f"First_center: {centers[0]}")
    #for other k-1 centers
    for cent in range (k-1):
        print(f"Starting center number {cent+1}")
        distance_matric = dict()
        distance_matric["CUST_ID"] = []
        distance_matric["Distance"] = []
#         for row in range (0, 200):
        for row in range (0, 8950):
            if row % 90 == 0:
                print(f"for {1+cent}th center: {row//89.50} %", end="\r")
            current_row = df[row:row+1]
            if current_row["CUST_ID"].values[0] not in centers:
                total_distance = 0
        
                for center_id in centers:
                    total_distance += distance(current_row, df[df["CUST_ID"] == center_id])[0]
                    
                distance_matric["CUST_ID"].append(current_row["CUST_ID"].values[0])
                distance_matric["Distance"].append(total_distance)
        
        distance_matric = pd.DataFrame(distance_matric)
#         print(distance_matric)
        next_center = distance_matric[distance_matric["Distance"] == max(distance_matric["Distance"])]["CUST_ID"].values[0]
        centers.append(next_center)
        max_distance = max(distance_matric["Distance"])
        print(f"Next center is {next_center} with {max_distance} distance")
        
    return centers

def find_clusters(center_list, df):
    #dict for each center: elements in clusters
    clusters = dict()
    for i in center_list:
        clusters[i] = []
        
    for point in df["CUST_ID"]:
        if point not in center_list:
            current_distance = 100000
            for center in center_list:
                dist = distance(df[df["CUST_ID"] == center], df[df["CUST_ID"] == point])
                if dist < current_distance:
                    current_distance = dist
                    temp_center = center
            
            clusters[temp_center].append([point, dist[0]])
    return clusters

In [27]:
with open("data.csv", "r") as file:
    data = pd.read_csv(file)
print(type(data))

<class 'pandas.core.frame.DataFrame'>


In [28]:
column_names = []
for col in data.columns:
    column_names.append(col)
    
column_names.pop(0)
print(column_names)

['BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES', 'ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE', 'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY', 'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY', 'CASH_ADVANCE_TRX', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS', 'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE']


In [29]:
x = data.drop("CUST_ID", axis=1)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x.values)
normalized_data = pd.DataFrame(x_scaled, columns = ['BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES', 'ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE', 'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY', 'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY', 'CASH_ADVANCE_TRX', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS', 'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE'])
normalized_data["CUST_ID"] = data["CUST_ID"]

In [31]:
normalized_data

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE,CUST_ID
0,0.002148,0.818182,0.001945,0.000000,0.004240,0.000000,0.166667,0.000000,0.083333,0.000000,0.00000,0.005587,0.033333,0.003979,0.001826,0.000000,1.0,C10001
1,0.168169,0.909091,0.000000,0.000000,0.000000,0.136685,0.000000,0.000000,0.000000,0.166667,0.03252,0.000000,0.233333,0.080893,0.014035,0.222222,1.0,C10002
2,0.131026,1.000000,0.015766,0.018968,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.00000,0.033520,0.250000,0.012264,0.008210,0.000000,1.0,C10003
3,0.087521,0.636364,0.030567,0.036775,0.000000,0.004366,0.083333,0.083333,0.000000,0.055555,0.00813,0.002793,0.250000,0.000000,0.000000,0.000000,1.0,C10004
4,0.042940,1.000000,0.000326,0.000393,0.000000,0.000000,0.083333,0.083333,0.000000,0.000000,0.00000,0.002793,0.040000,0.013374,0.003204,0.000000,1.0,C10005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8945,0.001496,1.000000,0.005936,0.000000,0.012939,0.000000,1.000000,0.000000,0.833333,0.000000,0.00000,0.016760,0.033333,0.006419,0.000640,0.500000,0.0,C19186
8946,0.001007,1.000000,0.006118,0.000000,0.013333,0.000000,1.000000,0.000000,0.833333,0.000000,0.00000,0.016760,0.033333,0.005439,0.000000,0.000000,0.0,C19187
8947,0.001229,0.833333,0.002945,0.000000,0.006418,0.000000,0.833333,0.000000,0.666667,0.000000,0.00000,0.013966,0.033333,0.001602,0.001079,0.250000,0.0,C19188
8948,0.000707,0.833333,0.000000,0.000000,0.000000,0.000776,0.000000,0.000000,0.000000,0.111111,0.01626,0.000000,0.016667,0.001036,0.000730,0.250000,0.0,C19189


In [37]:
centers = find_centers(normalized_data, 10)
clusters = find_clusters(centers, normalized_data)

First_center: C14442
Starting center number 1
Next center is C11300 with 2.259359985008784 distance
Starting center number 2
Next center is C15349 with 4.743820844172198 distance
Starting center number 3
Next center is C12226 with 6.8696106177150895 distance
Starting center number 4
Next center is C10574 with 8.712280305562345 distance
Starting center number 5
Next center is C13663 with 11.278317749752244 distance
Starting center number 6
Next center is C17430 with 12.66999758836119 distance
Starting center number 7
Next center is C14499 with 14.913127151180898 distance
Starting center number 8
Next center is C18536 with 16.90051775412315 distance
Starting center number 9
Next center is C11657 with 18.850609649982232 distance


In [38]:
print(clusters.keys())

dict_keys(['C14442', 'C11300', 'C15349', 'C12226', 'C10574', 'C13663', 'C17430', 'C14499', 'C18536', 'C11657'])


In [40]:
for i in clusters:
    for j in clusters[i]:
        print(j[0], end=" ")


C10001 C10003 C10005 C10006 C10008 C10009 C10011 C10012 C10013 C10014 C10015 C10016 C10018 C10019 C10020 C10021 C10022 C10023 C10024 C10026 C10027 C10028 C10029 C10030 C10031 C10032 C10033 C10034 C10035 C10036 C10038 C10043 C10044 C10045 C10046 C10047 C10050 C10051 C10052 C10053 C10054 C10056 C10057 C10058 C10059 C10060 C10061 C10062 C10063 C10067 C10068 C10071 C10072 C10073 C10074 C10077 C10078 C10079 C10080 C10081 C10082 C10083 C10084 C10086 C10088 C10090 C10092 C10093 C10095 C10096 C10097 C10098 C10099 C10100 C10101 C10106 C10107 C10108 C10109 C10110 C10111 C10113 C10114 C10115 C10116 C10117 C10119 C10120 C10121 C10125 C10126 C10127 C10128 C10131 C10133 C10137 C10138 C10140 C10141 C10142 C10143 C10145 C10146 C10147 C10149 C10150 C10151 C10152 C10153 C10154 C10155 C10156 C10157 C10158 C10159 C10160 C10161 C10162 C10163 C10164 C10165 C10167 C10168 C10169 C10170 C10171 C10172 C10173 C10174 C10175 C10178 C10180 C10181 C10182 C10184 C10185 C10186 C10190 C10191 C10192 C10193 C10194 C10196

 C12720 C12721 C12722 C12723 C12724 C12725 C12726 C12728 C12730 C12731 C12732 C12733 C12735 C12736 C12738 C12739 C12740 C12741 C12742 C12743 C12746 C12747 C12748 C12749 C12750 C12751 C12754 C12755 C12756 C12757 C12758 C12760 C12761 C12762 C12763 C12764 C12765 C12766 C12769 C12771 C12773 C12774 C12775 C12776 C12777 C12778 C12779 C12780 C12781 C12782 C12784 C12787 C12788 C12789 C12792 C12793 C12795 C12796 C12797 C12799 C12801 C12802 C12806 C12807 C12809 C12810 C12811 C12812 C12814 C12815 C12819 C12822 C12823 C12824 C12825 C12826 C12827 C12829 C12830 C12831 C12834 C12836 C12837 C12838 C12839 C12840 C12842 C12843 C12844 C12846 C12847 C12849 C12851 C12852 C12853 C12854 C12857 C12858 C12860 C12861 C12862 C12863 C12864 C12865 C12866 C12868 C12872 C12874 C12876 C12877 C12879 C12880 C12881 C12885 C12886 C12888 C12889 C12890 C12891 C12892 C12893 C12894 C12895 C12896 C12899 C12901 C12903 C12904 C12906 C12908 C12909 C12913 C12914 C12916 C12917 C12919 C12921 C12922 C12924 C12925 C12926 C12927 C1292

 C15256 C15257 C15258 C15260 C15261 C15263 C15264 C15265 C15266 C15267 C15269 C15270 C15271 C15272 C15274 C15275 C15276 C15277 C15278 C15280 C15282 C15285 C15287 C15288 C15289 C15290 C15291 C15292 C15293 C15294 C15295 C15296 C15300 C15302 C15304 C15305 C15306 C15309 C15310 C15312 C15313 C15314 C15317 C15318 C15319 C15320 C15321 C15322 C15324 C15326 C15327 C15328 C15330 C15331 C15332 C15333 C15334 C15335 C15338 C15339 C15341 C15342 C15343 C15344 C15346 C15347 C15351 C15352 C15353 C15354 C15355 C15358 C15359 C15361 C15362 C15364 C15368 C15369 C15370 C15372 C15373 C15374 C15376 C15377 C15378 C15380 C15381 C15382 C15383 C15384 C15385 C15386 C15387 C15388 C15389 C15392 C15393 C15394 C15395 C15396 C15397 C15398 C15399 C15401 C15402 C15403 C15404 C15405 C15406 C15407 C15409 C15410 C15411 C15415 C15417 C15420 C15421 C15423 C15424 C15425 C15426 C15428 C15429 C15431 C15432 C15435 C15437 C15438 C15439 C15440 C15442 C15445 C15446 C15448 C15449 C15451 C15452 C15453 C15454 C15457 C15459 C15460 C1546

 C17427 C17429 C17432 C17435 C17436 C17437 C17438 C17439 C17440 C17441 C17442 C17443 C17444 C17446 C17448 C17450 C17451 C17453 C17455 C17456 C17458 C17459 C17460 C17461 C17462 C17463 C17465 C17466 C17467 C17468 C17470 C17474 C17476 C17478 C17479 C17480 C17481 C17482 C17483 C17484 C17485 C17486 C17489 C17490 C17491 C17493 C17495 C17496 C17497 C17498 C17500 C17501 C17502 C17503 C17504 C17505 C17506 C17508 C17509 C17512 C17513 C17514 C17516 C17517 C17518 C17519 C17522 C17523 C17526 C17528 C17530 C17532 C17534 C17535 C17538 C17539 C17540 C17541 C17544 C17545 C17546 C17547 C17548 C17549 C17551 C17553 C17554 C17555 C17556 C17557 C17558 C17559 C17560 C17561 C17566 C17567 C17569 C17570 C17571 C17572 C17573 C17574 C17577 C17578 C17579 C17580 C17583 C17585 C17586 C17587 C17588 C17589 C17590 C17591 C17593 C17594 C17595 C17599 C17600 C17602 C17603 C17604 C17606 C17608 C17609 C17610 C17611 C17614 C17616 C17618 C17620 C17621 C17623 C17624 C17626 C17628 C17630 C17632 C17634 C17635 C17636 C17638 C1763

C10004 C10010 C10017 C10040 C10041 C10048 C10065 C10069 C10075 C10085 C10104 C10105 C10132 C10135 C10136 C10176 C10179 C10187 C10189 C10199 C10207 C10213 C10223 C10231 C10254 C10286 C10288 C10319 C10327 C10347 C10354 C10365 C10366 C10370 C10384 C10387 C10396 C10398 C10411 C10427 C10456 C10488 C10498 C10504 C10540 C10546 C10559 C10567 C10573 C10597 C10610 C10621 C10646 C10662 C10680 C10683 C10685 C10694 C10698 C10707 C10714 C10723 C10728 C10741 C10751 C10758 C10763 C10764 C10767 C10771 C10777 C10781 C10792 C10804 C10810 C10816 C10823 C10825 C10831 C10844 C10850 C10860 C10872 C10910 C10923 C10928 C10963 C10971 C10974 C10981 C10989 C10990 C11044 C11052 C11056 C11063 C11069 C11075 C11076 C11079 C11101 C11107 C11110 C11112 C11133 C11161 C11166 C11171 C11185 C11198 C11200 C11209 C11217 C11219 C11226 C11232 C11233 C11235 C11248 C11253 C11254 C11259 C11272 C11279 C11281 C11287 C11295 C11297 C11298 C11304 C11311 C11328 C11340 C11352 C11354 C11379 C11402 C11432 C11467 C11473 C11477 C11517 C11530

C10002 C10025 C10037 C10039 C10049 C10064 C10070 C10094 C10102 C10112 C10118 C10122 C10124 C10130 C10134 C10139 C10148 C10166 C10183 C10188 C10296 C10304 C10315 C10324 C10326 C10333 C10335 C10345 C10380 C10381 C10400 C10412 C10419 C10423 C10444 C10445 C10451 C10457 C10458 C10484 C10489 C10490 C10505 C10508 C10513 C10517 C10519 C10535 C10539 C10550 C10566 C10578 C10582 C10600 C10604 C10607 C10618 C10626 C10627 C10640 C10667 C10701 C10706 C10708 C10712 C10729 C10734 C10737 C10743 C10748 C10754 C10759 C10762 C10766 C10769 C10783 C10785 C10788 C10791 C10793 C10809 C10815 C10817 C10818 C10829 C10832 C10847 C10861 C10876 C10879 C10884 C10896 C10903 C10909 C10914 C10934 C10942 C10955 C10957 C10961 C10976 C10996 C11018 C11023 C11064 C11065 C11072 C11111 C11121 C11123 C11130 C11136 C11140 C11148 C11157 C11173 C11206 C11222 C11224 C11225 C11239 C11261 C11273 C11280 C11288 C11289 C11303 C11326 C11360 C11373 C11377 C11378 C11381 C11386 C11390 C11394 C11411 C11424 C11433 C11435 C11437 C11453 C11456